CAUTION! Execute Classification_preprocessing.ipynb first!

In [1]:
!pip install pandas
!pip install openml
!pip install nbconvert
!brew install pandoc
!pip install seaborn
!pip install sklearn

You should consider upgrading via the '/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/bin/python -m pip install --upgrade pip' command.
Running `brew update --auto-update`...
Installing from the API is now the default behaviour!
You can save space and time by running:
  brew untap homebrew/core
  brew untap homebrew/cask
==> Auto-updated Homebrew!
Updated 2 taps (homebrew/cask-versions and gcenx/wine).
==> New Casks
forklift3                  microsoft-office-2019      sublime-text3

You have 41 outdated formulae and 1 outdated cask installed.

pandoc 3.1.8 is already installed but outdated (so it will be upgraded).
==> Downloadin

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import math

from pprint import pprint
from sklearn.preprocessing import minmax_scale, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate

from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score

# import you metrics here
from sklearn.metrics import f1_score, balanced_accuracy_score, precision_score, recall_score

from timeit import default_timer as timer


import warnings


# Load pre-processed data

## Dataset 1: Biomed data on muscular dystropy


In [3]:
with open('data/biomed/preprocessed_biomed_data.pickle', 'rb') as handle:
    x_biomed_train = pickle.load(handle)
    x_biomed_train_preprocessed = pickle.load(handle)
    x_biomed_train_preprocessed_unscaled = pickle.load(handle)
    y_biomed_train = pickle.load(handle)
    x_biomed_test = pickle.load(handle)
    x_biomed_test_preprocessed = pickle.load(handle)
    x_biomed_test_preprocessed_unscaled = pickle.load(handle)
    y_biomed_test = pickle.load(handle)

## Dataset 2: Fertility

In [4]:
with open('data/fertility/preprocessed_fertility_data.pickle', 'rb') as handle:
    x_fert_train = pickle.load(handle)
    x_fert_test = pickle.load(handle)
    y_fert_train = pickle.load(handle)
    y_fert_test = pickle.load(handle)

## Dataset 3: Amazon Reviews

In [5]:
with open('data/reviews/preprocessed_reviews_data.pickle', 'rb') as handle:
        x_reviews_train = pickle.load(handle)
        x_reviews_train_preprocessed = pickle.load(handle)
        x_reviews_train_preprocessed_unscaled = pickle.load(handle)
        y_reviews_train = pickle.load(handle)
        x_reviews_test_preprocessed = pickle.load(handle)
        x_reviews_test_preprocessed_unscaled = pickle.load(handle)
        reviews_test_ID = pickle.load(handle)

## Dataset 4: Congress

In [6]:
with open('data/congress/preprocessed_congress_data.pickle', 'rb') as handle:
        x_congress_train = pickle.load(handle)
        x_congress_train_preprocessed = pickle.load(handle)
        y_congress_train = pickle.load(handle)
        x_congress_test = pickle.load(handle)
        x_congress_test_preprocessed = pickle.load(handle)
        congress_test_ID = pickle.load(handle)

In [7]:
# general preprocessor
# with MinMaxScaler
preprocessor = Pipeline(
   steps=[
       ("imputation", KNNImputer(n_neighbors=2, weights="uniform")),
       ("scaler", MinMaxScaler())
    ]
)
# without scaling
preprocessor_no_scaling = Pipeline(
   steps=[
       ("imputation", KNNImputer(n_neighbors=2, weights="uniform"))
    ]
)

In [8]:
# shuffling two objects in the same way
# input: two pandas objects whose orders correspond
# output: the input objects, but with shuffled order, but their orders still correspond
def shuffle(*objects):
    np.random.seed(42)
    new_index = np.random.permutation(objects[0].index)
    objects = [obj.reindex(new_index) for obj in objects]
    if len(objects) == 1: return objects[0]
    return objects


## Dataset 1: Biomed data on muscular dystropy

In [9]:
with open('data/biomed/x_biomed.pickle', 'rb') as handle:
    x_biomed = pickle.load(handle)
with open('data/biomed/y_biomed.pickle', 'rb') as handle:
    y_biomed = pickle.load(handle)

In [10]:
print(y_biomed)
# recoding the labels to be 0 or 1
# 0: normal
# 1: carrier
y_biomed.replace({"carrier": 1, "normal": 0}, inplace=True)
y_biomed

0      carrier
1      carrier
2      carrier
3      carrier
4      carrier
        ...   
204     normal
205     normal
206     normal
207     normal
208     normal
Name: class, Length: 209, dtype: category
Categories (2, object): ['carrier' < 'normal']


0      1
1      1
2      1
3      1
4      1
      ..
204    0
205    0
206    0
207    0
208    0
Name: class, Length: 209, dtype: category
Categories (2, int64): [1 < 0]

In [11]:
x_biomed = x_biomed.drop(["Observation_number", "Hospital_identification_number_for_blood_sample", "Date_that_blood_sample_was_taken"], 
                         axis=1)
x_biomed

Age_of_patient     ml     m2    m3     m4
0                30  167.0   89.0  25.6  364.0
1                41  104.0   81.0  26.8  245.0
2                22   30.0  108.0   8.8  284.0
3                22   44.0  104.0  17.4  172.0
4                20   65.0   87.0  23.8  198.0
..              ...    ...    ...   ...    ...
204              32   41.0   78.5  10.9  191.0
205              32   43.0   87.5   6.0  136.0
206              32   30.0   90.5  15.3  136.0
207              33   30.0   85.0  11.4  176.0
208              33   43.0   88.5  20.3  175.0

[209 rows x 5 columns]

In [12]:
# shuffle the data
x_biomed, y_biomed = shuffle(x_biomed, y_biomed)

In [13]:
# data splitting
# first join x and y back together
x_biomed["y"]=y_biomed
# split the whole data into train and test set
x_biomed_train, x_biomed_test = train_test_split(x_biomed, test_size=0.2, random_state=0, stratify=x_biomed.y)
# separate the class labels to be their own series again
y_biomed_train = x_biomed_train.pop("y")
y_biomed_test = x_biomed_test.pop("y")

In [14]:
# in total 15 rows have NA values in either m2 or o4
# apply preprocessing, to both the test and training set
x_biomed_train_preprocessed = pd.DataFrame(preprocessor.fit_transform(x_biomed_train))
x_biomed_train_preprocessed.columns = x_biomed.columns[:-1]
x_biomed_train_preprocessed

x_biomed_test_preprocessed = pd.DataFrame(preprocessor.fit_transform(x_biomed_test))
x_biomed_test_preprocessed.columns = x_biomed.columns[:-1]




x_biomed_train_preprocessed_unscaled = pd.DataFrame(preprocessor_no_scaling.fit_transform(x_biomed_train))
x_biomed_train_preprocessed_unscaled.columns = x_biomed.columns[:-1]

x_biomed_test_preprocessed_unscaled = pd.DataFrame(preprocessor_no_scaling.fit_transform(x_biomed_test))
x_biomed_test_preprocessed_unscaled.columns = x_biomed.columns[:-1]


In [15]:
print(
    "Training features: ", x_biomed_train.shape, "\n",
    "Training labels: ", y_biomed_train.shape, "\n",
    "Test features: ", x_biomed_test.shape, "\n",
    "Test labels: ", y_biomed_test.shape, "\n",
    sep=""
)

Training features: (167, 5)
Training labels: (167,)
Test features: (42, 5)
Test labels: (42,)



## Dataset 2: Fertility

In [16]:
with open('data/fertility/x_fert.pickle', 'rb') as handle:
    x_fert = pickle.load(handle)
with open('data/fertility/y_fert.pickle', 'rb') as handle:
    y_fert = pickle.load(handle)

# shuffle the data, just in case
x_fert, y_fert = shuffle(x_fert, y_fert)

In [17]:
y_fert

83    0
53    0
70    1
45    0
44    0
     ..
60    0
71    0
14    0
92    0
51    0
Name: fertility, Length: 100, dtype: category
Categories (2, int64): [0 < 1]

In [18]:
x_fert.columns

Index(['season', 'age', 'child_diseases', 'accident', 'surgery', 'high_fever',
       'alcohol', 'smoking', 'hours_sitting', 'fall', 'spring', 'summer',
       'winter', 'fever_never', 'fever_not_recent', 'fever_recent',
       'smoking_daily', 'smoking_never', 'smoking_occasionally',
       'alcohol_daily', 'alcohol_rarely_or_never', 'alcohol_several_daily',
       'alcohol_several_weekly', 'alcohol_weekly'],
      dtype='object')

In [19]:
# drop all columns that are not numeric or 1-hot
x_fert = x_fert.drop(["season", "high_fever", "alcohol", "smoking"], axis=1)

In [20]:
x_fert

age  child_diseases  accident  surgery  hours_sitting  fall  spring  \
83  0.86               0         0        0           0.25     0       1   
53  0.58               0         0        0           0.19     0       1   
70  0.50               0         0        1           0.88     0       1   
45  0.53               0         1        1           0.44     0       0   
44  0.53               0         0        1           0.31     0       0   
..   ...             ...       ...      ...            ...   ...     ...   
60  0.56               1         1        1           1.00     1       0   
71  0.69               0         1        1           0.31     0       0   
14  0.94               0         0        0           0.25     1       0   
92  0.75               0         0        1           0.38     0       0   
51  0.53               0         0        1           0.75     0       1   

    summer  winter  fever_never  fever_not_recent  fever_recent  \
83       0       0            1                 0             0   
53       0       0            0                 0             1   
70       0       0            0                 0             1   
45       0       1            1                 0             0   
44       0       1            1                 0             0   
..     ...     ...          ...               ...           ...   
60       0       0            1                 0             0   
71       1       0            1                 0             0   
14       0       0            0                 1             0   
92       1       0            0                 1             0   
51       0       0            1                 0             0   

    smoking_daily  smoking_never  smoking_occasionally  alcohol_daily  \
83              0              1                     0              0   
53              0              0                     1              0   
70              0              0                     1              0   
45              0              0                     1              0   
44              0              0                     1              0   
..            ...            ...                   ...            ...   
60              0              0                     1              0   
71              0              1                     0              0   
14              0              1                     0              0   
92              0              1                     0              0   
51              0              1                     0              0   

    alcohol_rarely_or_never  alcohol_several_daily  alcohol_several_weekly  \
83                        1                      0                       0   
53                        0                      0                       0   
70                        0                      0                       0   
45                        1                      0                       0   
44                        1                      0                       0   
..                      ...                    ...                     ...   
60                        0                      0                       0   
71                        1                      0                       0   
14                        0                      1                       0   
92                        0                      0                       0   
51                        1                      0                       0   

    alcohol_weekly  
83               0  
53               1  
70               1  
45               0  
44               0  
..             ...  
60               1  
71               0  
14               0  
92               1  
51               0  

[100 rows x 20 columns]

In [21]:
x_fert.describe()

age  child_diseases    accident     surgery  hours_sitting  \
count  100.000000      100.000000  100.000000  100.000000     100.000000   
mean     0.669000        0.130000    0.560000    0.490000       0.406800   
std      0.121319        0.337998    0.498888    0.502418       0.186395   
min      0.500000        0.000000    0.000000    0.000000       0.060000   
25%      0.560000        0.000000    0.000000    0.000000       0.250000   
50%      0.670000        0.000000    1.000000    0.000000       0.380000   
75%      0.750000        0.000000    1.000000    1.000000       0.500000   
max      1.000000        1.000000    1.000000    1.000000       1.000000   

             fall      spring      summer      winter  fever_never  \
count  100.000000  100.000000  100.000000  100.000000   100.000000   
mean     0.310000    0.370000    0.040000    0.280000     0.280000   
std      0.464823    0.485237    0.196946    0.451261     0.451261   
min      0.000000    0.000000    0.000000    0.000000     0.000000   
25%      0.000000    0.000000    0.000000    0.000000     0.000000   
50%      0.000000    0.000000    0.000000    0.000000     0.000000   
75%      1.000000    1.000000    0.000000    1.000000     1.000000   
max      1.000000    1.000000    1.000000    1.000000     1.000000   

       fever_not_recent  fever_recent  smoking_daily  smoking_never  \
count        100.000000    100.000000      100.00000     100.000000   
mean           0.630000      0.090000        0.21000       0.560000   
std            0.485237      0.287623        0.40936       0.498888   
min            0.000000      0.000000        0.00000       0.000000   
25%            0.000000      0.000000        0.00000       0.000000   
50%            1.000000      0.000000        0.00000       1.000000   
75%            1.000000      0.000000        0.00000       1.000000   
max            1.000000      1.000000        1.00000       1.000000   

       smoking_occasionally  alcohol_daily  alcohol_rarely_or_never  \
count            100.000000         100.00               100.000000   
mean               0.230000           0.01                 0.400000   
std                0.422953           0.10                 0.492366   
min                0.000000           0.00                 0.000000   
25%                0.000000           0.00                 0.000000   
50%                0.000000           0.00                 0.000000   
75%                0.000000           0.00                 1.000000   
max                1.000000           1.00                 1.000000   

       alcohol_several_daily  alcohol_several_weekly  alcohol_weekly  
count                 100.00              100.000000      100.000000  
mean                    0.01                0.190000        0.390000  
std                     0.10                0.394277        0.490207  
min                     0.00                0.000000        0.000000  
25%                     0.00                0.000000        0.000000  
50%                     0.00                0.000000        0.000000  
75%                     0.00                0.000000        1.000000  
max                     1.00                1.000000        1.000000

now everything is 1-hot-encoded

In [22]:
# data splitting
# first join x and y back together
x_fert["y"]=y_fert
# split the whole data into train and test set
x_fert_train, x_fert_test = train_test_split(x_fert, test_size=0.2, random_state=0, stratify=x_fert.y)
# separate the class labels to be their own series again
y_fert_train = x_fert_train.pop("y")
y_fert_test = x_fert_test.pop("y")

In [23]:
print(
    "Training features: ", x_fert_train.shape, "\n",
    "Training labels: ", y_fert_train.shape, "\n",
    "Test features: ", x_fert_test.shape, "\n",
    "Test labels: ", y_fert_test.shape, "\n",
    sep=""
)

Training features: (80, 20)
Training labels: (80,)
Test features: (20, 20)
Test labels: (20,)



## Dataset 3: Amazon Reviews

In [24]:
reviews_train = pd.read_csv('data/reviews/amazon_review_ID.shuf.lrn.csv')  
reviews_test = pd.read_csv('data/reviews/amazon_review_ID.shuf.tes.csv')  

In [25]:
y_reviews_train = reviews_train.Class

In [26]:
y_reviews_train.nunique()

50

In [27]:
y_reviews_train.value_counts()

Class
Calvinnme      21
Walters        21
Nigam          20
Wilson         20
Davisson       19
Chachra        19
Sherwin        18
Vernon         18
Kolln          18
Power          18
Lee            17
Brody          17
Koenig         17
Neal           17
Hayes          17
Harp           17
CFH            17
Auken          17
Brown          16
Grove          16
Engineer       15
Lawyeraau      15
Shea           15
Mitchell       15
Mark           15
Cutey          15
Chell          14
Taylor         14
Peterson       14
Ashbacher      14
Cholette       14
Janson         14
Chandler       14
Mahlers2nd     14
Riley          14
Agresti        13
Messick        13
Lovitt         13
Bukowsky       12
Vision         12
Corn           12
Robert         12
Merritt        12
Blankenship    12
Dent           11
Johnson        11
Goonan         11
Morrison       11
Comdet         10
McKee           9
Name: count, dtype: int64

In [28]:
x_reviews_train = reviews_train.drop(["ID", "Class"], axis=1)
x_reviews_train

V1  V2  V3  V4  V5  V6  V7  V8  V9  V10  ...  V9991  V9992  V9993  V9994  \
0    14   9   8  12   7   4   5   3   2    2  ...      2      0      1      0   
1    12   3   8   6   5   3   2   3   2    0  ...      0      0      0      0   
2    17   5   7  12   8   3   0   3   2    3  ...      0      0      1      0   
3     9   6  10   6   4   7   7   3   3    3  ...      0      0      0      0   
4     8   4   5   5   1   7   0   0   0    0  ...      0      0      0      3   
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...    ...    ...    ...    ...   
745   9   4   3   4   6   7   2   1   0    1  ...      0      1      1      1   
746  13   6   1  13   2   5   7   5   3    1  ...      0      0      0      0   
747  19   9  12   3   9   7   5   2   5    2  ...      2      1      0      0   
748   5  14  16   7   2  12   9   3   7    4  ...      0      0      0      0   
749   7  10   4   4   9   0   2   8   1    2  ...      0      0      0      1   

     V9995  V9996  V9997  V9998  V9999  V10000  
0        1      0      1      1      0       0  
1        0      0      0      0      0       0  
2        0      0      0      1      0       0  
3        0      2      0      0      0       0  
4        0      0      0      0      0       0  
..     ...    ...    ...    ...    ...     ...  
745      2      0      0      0      0       0  
746      1      0      0      0      1       0  
747      0      0      0      0      1       0  
748      0      0      0      0      3       0  
749      0      0      0      0      1       0  

[750 rows x 10000 columns]

In [29]:
x_reviews_train.isna().sum().sum()

0

In [30]:
reviews_test_ID = reviews_test.pop("ID")
reviews_test

V1  V2  V3  V4  V5  V6  V7  V8  V9  V10  ...  V9991  V9992  V9993  V9994  \
0    12   8   9  10   9   4   0   3   2    1  ...      0      0      0      0   
1     7   6   2   5   1   7   9   5   4   11  ...      4      0      0      0   
2     9   7   3   7   5   3   1   2   5    1  ...      0      0      0      0   
3     7   7   6   6   5   2  14   0   9    8  ...      0      0      0      0   
4    26  13  11  15   6   4   3   5   4    1  ...      0      1      0      1   
..   ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...    ...    ...    ...    ...   
745  14   5   4  11   7   4   2   3   2    0  ...      0      0      0      0   
746  13  13   6   3  12   5   4   2   4   10  ...      0      0      1      0   
747  12   9   2   4   2   4   2   8   5    1  ...      0      0      0      0   
748   7   5   4   7  10   2   1   0   0    2  ...      0      0      0      0   
749  12   5   4   8   5   2   2   0   1    2  ...      0      1      0      0   

     V9995  V9996  V9997  V9998  V9999  V10000  
0        0      0      0      0      0       0  
1        0      0      0      0      0       0  
2        0      1      0      1      0       0  
3        0      0      0      0      1       0  
4        0      0      0      1      1       0  
..     ...    ...    ...    ...    ...     ...  
745      0      1      0      0      0       0  
746      0      1      0      0      2       0  
747      0      0      0      0      0       0  
748      0      0      0      0      0       0  
749      0      0      0      0      0       0  

[750 rows x 10000 columns]

all variables appear to be some sort of rating, I guess either number of views (streams/clicks) or a certain number given as a rating. its not a likert rating, as the numbers per product even go up to over 30. 
if we had any evidence that the ratings were a fixed scale we could use the numbers as is, but since we do ot have that I guess it might be wise to scale the data (but make the step optional just in case)

Im doing mix-max-scaling
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html#sklearn.preprocessing.minmax_scale

In [31]:
# shuffle the training data
x_reviews_train, y_reviews_train = shuffle(x_reviews_train, y_reviews_train)


In [32]:
# preprocessing

# apply the preprocessing pipeline, to both the training and test data
x_reviews_train_preprocessed = pd.DataFrame(preprocessor.fit_transform(x_reviews_train))
# salvage the column names
x_reviews_train_preprocessed.columns = x_reviews_train.columns
x_reviews_train_preprocessed

# apply the preprocessing pipeline, to both the training and test data
x_reviews_test_preprocessed = pd.DataFrame(preprocessor.fit_transform(reviews_test))
# salvage the column names
x_reviews_test_preprocessed.columns = reviews_test.columns


# preview
x_reviews_train_preprocessed

V1        V2    V3    V4        V5        V6        V7        V8  \
0    0.28125  0.428571  0.40  0.08  0.291667  0.333333  0.000000  0.500000   
1    0.40625  0.523810  0.25  0.52  0.166667  0.200000  0.111111  0.214286   
2    0.53125  0.285714  0.20  0.44  0.458333  0.333333  0.611111  0.500000   
3    0.50000  0.428571  0.90  0.68  0.083333  0.666667  0.000000  0.571429   
4    0.09375  0.190476  0.15  0.04  0.208333  0.133333  0.500000  0.071429   
..       ...       ...   ...   ...       ...       ...       ...       ...   
745  0.18750  0.190476  0.05  0.08  0.166667  0.066667  0.000000  0.000000   
746  0.37500  0.428571  0.30  0.40  0.250000  0.400000  0.000000  0.285714   
747  0.34375  0.095238  0.10  0.20  0.208333  0.133333  0.333333  0.214286   
748  0.28125  0.571429  0.55  0.28  0.125000  0.533333  0.388889  0.214286   
749  0.78125  0.333333  0.10  0.52  0.291667  0.466667  0.000000  0.285714   

           V9       V10  ...  V9991     V9992  V9993  V9994  V9995  V9996  \
0    0.230769  0.263158  ...   0.00  0.000000   0.00    0.0    0.0   0.00   
1    0.153846  0.000000  ...   0.25  0.000000   0.00    0.0    0.0   0.00   
2    0.692308  0.105263  ...   0.00  0.000000   0.00    0.2    0.0   0.00   
3    0.384615  0.105263  ...   0.00  0.000000   0.00    0.0    0.0   0.25   
4    0.307692  0.052632  ...   0.25  0.000000   0.00    0.0    0.0   0.00   
..        ...       ...  ...    ...       ...    ...    ...    ...    ...   
745  0.076923  0.052632  ...   0.00  0.000000   0.00    0.0    0.0   0.00   
746  0.076923  0.105263  ...   0.00  0.000000   0.25    0.0    0.0   0.00   
747  0.384615  0.052632  ...   0.00  0.166667   0.00    0.0    0.0   0.25   
748  0.384615  0.210526  ...   0.00  0.000000   0.00    0.0    0.0   0.00   
749  0.153846  0.157895  ...   0.00  0.000000   0.25    0.0    0.0   0.00   

        V9997     V9998     V9999  V10000  
0    0.000000  0.000000  0.000000     0.0  
1    0.000000  0.000000  0.333333     0.0  
2    0.333333  0.000000  0.000000     0.0  
3    0.000000  0.000000  0.000000     0.0  
4    0.000000  0.333333  0.000000     0.0  
..        ...       ...       ...     ...  
745  0.000000  0.000000  0.000000     0.0  
746  0.333333  0.000000  0.000000     0.0  
747  0.000000  0.000000  0.000000     0.0  
748  0.000000  0.000000  0.333333     0.0  
749  0.000000  0.000000  0.000000     0.0  

[750 rows x 10000 columns]

In [33]:
print(
    "Training features: ", x_reviews_train_preprocessed.shape, "\n",
    "Training labels: ", y_reviews_train.shape, "\n",
    "Test features: ", x_reviews_test_preprocessed.shape, "\n",
    sep=""
)

Training features: (750, 10000)
Training labels: (750,)
Test features: (750, 10000)



## Dataset 4: Congress

In [34]:
congress_train = pd.read_csv("data/congress/CongressionalVotingID.shuf.lrn.csv")
sample_solution = pd.read_csv("data/congress/CongressionalVotingID.shuf.sol.ex.csv")
x_congress_test = pd.read_csv("data/congress/CongressionalVotingID.shuf.tes.csv")


In [35]:
congress_train
#the test set looks the same, just smaller and no class column

ID       class handicapped-infants water-project-cost-sharing  \
0    249  republican             unknown                    unknown   
1    140    democrat                   y                          n   
2    383    democrat                   y                          y   
3    201    democrat                   n                          n   
4    297  republican                   n                          n   
..   ...         ...                 ...                        ...   
213  106    democrat                   y                          y   
214  271    democrat                   y                          y   
215  275  republican                   y                          n   
216  354  republican                   n                          n   
217  324    democrat                   y                          y   

    adoption-of-the-budget-resolution physician-fee-freeze el-salvador-aid  \
0                             unknown              unknown         unknown   
1                                   y                    n               n   
2                                   n                    y               y   
3                                   y                    n               n   
4                                   y                    y               y   
..                                ...                  ...             ...   
213                                 y                    n               n   
214                                 y                    n               n   
215                                 n                    y               y   
216                                 y                    y               y   
217                                 y                    n               y   

    religious-groups-in-schools anti-satellite-test-ban  \
0                       unknown                 unknown   
1                             y                       y   
2                             y                       n   
3                             n                       y   
4                             y                       n   
..                          ...                     ...   
213                           n                       n   
214                           n                       y   
215                           n                       y   
216                           y                       y   
217                           y                       n   

    aid-to-nicaraguan-contras mx-missile immigration  \
0                     unknown    unknown     unknown   
1                           y          y           n   
2                           n          n           n   
3                           y          y           n   
4                           n          n           y   
..                        ...        ...         ...   
213                         y          y           n   
214                         y          y           n   
215                         n          n           y   
216                         n          n           y   
217                         y    unknown           y   

    synfuels-crporation-cutback education-spending superfund-right-to-sue  \
0                       unknown            unknown                unknown   
1                             n                  n                      n   
2                             y                  n                      y   
3                             n                  n                      n   
4                             n                  y                      y   
..                          ...                ...                    ...   
213                           y                  n                      n   
214                           n                  n                      n   
215                           n                  n                      n   
216                           y                  y              

In [36]:
#Extracting independent variables
y_congress_train = congress_train['class']

In [37]:
#Extract dependent variable
x_congress_train = congress_train.drop(["ID", "class"], axis=1)
x_congress_test_ID = x_congress_test.pop("ID")
x_congress_train

handicapped-infants water-project-cost-sharing  \
0               unknown                    unknown   
1                     y                          n   
2                     y                          y   
3                     n                          n   
4                     n                          n   
..                  ...                        ...   
213                   y                          y   
214                   y                          y   
215                   y                          n   
216                   n                          n   
217                   y                          y   

    adoption-of-the-budget-resolution physician-fee-freeze el-salvador-aid  \
0                             unknown              unknown         unknown   
1                                   y                    n               n   
2                                   n                    y               y   
3                                   y                    n               n   
4                                   y                    y               y   
..                                ...                  ...             ...   
213                                 y                    n               n   
214                                 y                    n               n   
215                                 n                    y               y   
216                                 y                    y               y   
217                                 y                    n               y   

    religious-groups-in-schools anti-satellite-test-ban  \
0                       unknown                 unknown   
1                             y                       y   
2                             y                       n   
3                             n                       y   
4                             y                       n   
..                          ...                     ...   
213                           n                       n   
214                           n                       y   
215                           n                       y   
216                           y                       y   
217                           y                       n   

    aid-to-nicaraguan-contras mx-missile immigration  \
0                     unknown    unknown     unknown   
1                           y          y           n   
2                           n          n           n   
3                           y          y           n   
4                           n          n           y   
..                        ...        ...         ...   
213                         y          y           n   
214                         y          y           n   
215                         n          n           y   
216                         n          n           y   
217                         y    unknown           y   

    synfuels-crporation-cutback education-spending superfund-right-to-sue  \
0                       unknown            unknown                unknown   
1                             n                  n                      n   
2                             y                  n                      y   
3                             n                  n                      n   
4                             n                  y                      y   
..                          ...                ...                    ...   
213                           y                  n                      n   
214                           n                  n                      n   
215                           n                  n                      n   
216                           y                  y                      y   
217                           n                  n                      y   

       crime duty-free-exports export-administration-act-south-africa  
0    unknown           unknown         

In [38]:
y_congress_train.nunique()

2

In [39]:
y_congress_train.value_counts()

class
democrat      137
republican     81
Name: count, dtype: int64

In [40]:
x_congress_summary = x_congress_train.describe()
x_congress_summary

handicapped-infants water-project-cost-sharing  \
count                  218                        218   
unique                   3                          3   
top                      n                          n   
freq                   110                         99   

       adoption-of-the-budget-resolution physician-fee-freeze el-salvador-aid  \
count                                218                  218             218   
unique                                 3                    3               3   
top                                    y                    n               n   
freq                                 131                  130             110   

       religious-groups-in-schools anti-satellite-test-ban  \
count                          218                     218   
unique                           3                       3   
top                              y                       y   
freq                           131                     126   

       aid-to-nicaraguan-contras mx-missile immigration  \
count                        218        218         218   
unique                         3          3           3   
top                            y          y           n   
freq                         128        105         108   

       synfuels-crporation-cutback education-spending superfund-right-to-sue  \
count                          218                218                    218   
unique                           3                  3                      3   
top                              n                  n                      n   
freq                           134                125                    103   

       crime duty-free-exports export-administration-act-south-africa  
count    218               218                                    218  
unique     3                 3                                      3  
top        y                 n                                      y  
freq     112               108                                    130

In [41]:
total_unknown_rows = x_congress_train[x_congress_train.eq('unknown').any(axis=1)].shape[0]
print(total_unknown_rows)

107


In [42]:
# shuffling the data
x_congress_train, y_congress_train = shuffle(x_congress_train, y_congress_train)

# Replacing all "unknown" values with the mode of the given column
x_congress_train.replace('unknown', pd.NA, inplace=True)
x_congress_test.replace('unknown', pd.NA, inplace=True)
# Replacing 'y' and 'n' with 1 and 0
x_congress_train.replace({'y': 1, 'n': 0}, inplace=True)
x_congress_test.replace({'y': 1, 'n': 0}, inplace=True)

# I have no clue why I need to do this replacement twice
# the original data stores NAs as "<NA>", which sklearn imputator cannot handle
# np.nan it can handle though
# when I run the second line of the following two, they are not replaced for some reason
# BUT!!! if I go the other direction first, and then replace it backwards again, it works
x_congress_train.replace({np.nan: "<NA>"}, inplace=True)
x_congress_train.replace({"<NA>": np.nan}, inplace=True)
#
x_congress_test.replace({np.nan: "<NA>"}, inplace=True)
x_congress_test.replace({"<NA>": np.nan}, inplace=True)


# applying the preprocessor
x_congress_train_processed = pd.DataFrame(preprocessor.fit_transform(x_congress_train))
x_congress_test_processed = pd.DataFrame(preprocessor.fit_transform(x_congress_test))
# slavage the original column names
x_congress_train_processed.columns = x_congress_train.columns
x_congress_test_processed.columns = x_congress_test.columns
x_congress_train_processed


handicapped-infants  water-project-cost-sharing  \
0                    0.0                         1.0   
1                    1.0                         0.0   
2                    0.0                         0.0   
3                    0.0                         1.0   
4                    1.0                         0.0   
..                   ...                         ...   
213                  0.0                         1.0   
214                  0.0                         1.0   
215                  0.0                         0.0   
216                  1.0                         1.0   
217                  0.0                         1.0   

     adoption-of-the-budget-resolution  physician-fee-freeze  el-salvador-aid  \
0                                  1.0                   0.0              0.0   
1                                  0.0                   1.0              1.0   
2                                  0.0                   1.0              1.0   
3                                  1.0                   0.0              0.0   
4                                  1.0                   1.0              1.0   
..                                 ...                   ...              ...   
213                                1.0                   0.0              0.0   
214                                1.0                   0.0              1.0   
215                                0.0                   1.0              1.0   
216                                1.0                   0.0              1.0   
217                                0.0                   1.0              1.0   

     religious-groups-in-schools  anti-satellite-test-ban  \
0                            0.0                      1.0   
1                            0.0                      1.0   
2                            1.0                      0.0   
3                            1.0                      1.0   
4                            1.0                      1.0   
..                           ...                      ...   
213                          1.0                      0.0   
214                          1.0                      0.0   
215                          1.0                      0.0   
216                          1.0                      0.0   
217                          1.0                      0.0   

     aid-to-nicaraguan-contras  mx-missile  immigration  \
0                          1.0         0.0          0.0   
1                          0.0         0.0          1.0   
2                          0.0         0.0          1.0   
3                          1.0         1.0          0.0   
4                          1.0         0.0          1.0   
..                         ...         ...          ...   
213                        1.0         1.0          1.0   
214                        1.0         0.0          1.0   
215                        0.0         0.0          0.0   
216                        1.0         0.0          1.0   
217                        0.0         0.0          1.0   

     synfuels-crporation-cutback  education-spending  superfund-right-to-sue  \
0                            1.0                 0.0                     0.0   
1                            0.0                 0.0                     0.0   
2                            0.0                 1.0                     0.0   
3                            1.0                 1.0                     0.0   
4                            1.0                 1.0                     0.0   
..                           ...                 ...                     ...   
213                          0.0                 0.0                     1.0   
214                          1.0                 0.0                     1.0   
215                          0.0                 1.0                     1.0   
216                          1.0                 0.0                     1.0   
217                          0.0     

In [43]:
x_congress_test_processed

handicapped-infants  water-project-cost-sharing  \
0                    1.0                         0.0   
1                    0.0                         0.0   
2                    1.0                         1.0   
3                    1.0                         1.0   
4                    1.0                         0.0   
..                   ...                         ...   
212                  0.0                         0.0   
213                  1.0                         0.0   
214                  0.0                         0.0   
215                  1.0                         0.0   
216                  0.0                         1.0   

     adoption-of-the-budget-resolution  physician-fee-freeze  el-salvador-aid  \
0                                  1.0                   0.0              1.0   
1                                  0.0                   1.0              1.0   
2                                  1.0                   0.0              0.0   
3                                  1.0                   0.0              0.0   
4                                  1.0                   0.0              1.0   
..                                 ...                   ...              ...   
212                                1.0                   0.0              0.0   
213                                1.0                   0.0              0.0   
214                                1.0                   1.0              1.0   
215                                1.0                   0.0              0.0   
216                                0.0                   1.0              1.0   

     religious-groups-in-schools  anti-satellite-test-ban  \
0                            1.0                      1.0   
1                            1.0                      0.0   
2                            0.0                      1.0   
3                            1.0                      1.0   
4                            1.0                      0.0   
..                           ...                      ...   
212                          0.0                      1.0   
213                          0.0                      1.0   
214                          1.0                      1.0   
215                          0.0                      1.0   
216                          1.0                      0.0   

     aid-to-nicaraguan-contras  mx-missile  immigration  \
0                          0.0         0.0          0.0   
1                          0.0         0.0          1.0   
2                          1.0         1.0          0.0   
3                          1.0         1.0          1.0   
4                          0.0         0.0          0.0   
..                         ...         ...          ...   
212                        1.0         1.0          1.0   
213                        1.0         1.0          1.0   
214                        1.0         0.0          1.0   
215                        1.0         1.0          0.0   
216                        0.0         0.0          0.0   

     synfuels-crporation-cutback  education-spending  superfund-right-to-sue  \
0                            1.0                 0.0                     0.0   
1                            0.0                 1.0                     1.0   
2                            1.0                 0.0                     0.0   
3                            0.0                 0.0                     0.0   
4                            0.0                 0.0                     0.0   
..                           ...                 ...                     ...   
212                          1.0                 0.0                     0.0   
213                          0.0                 0.0                     0.0   
214                          0.0                 0.0                     0.0   
215                          0.0                 0.0                     0.0   
216                          0.0     

In [44]:
print("before")
print("NAs in training set:", x_congress_train.isna().sum().sum())
print("NAs in training set:", x_congress_test.isna().sum().sum())
print("after")
print("NAs in training set:", x_congress_train_processed.isna().sum().sum())
print("NAs in training set:", x_congress_test_processed.isna().sum().sum())

before
NAs in training set: 206
NAs in training set: 186
after
NAs in training set: 0
NAs in training set: 0


In [45]:
y_congress_train.describe()

count          218
unique           2
top       democrat
freq           137
Name: class, dtype: object

In [46]:
x_congress_summary.loc["freq"].max

<bound method Series.max of handicapped-infants                       110
water-project-cost-sharing                 99
adoption-of-the-budget-resolution         131
physician-fee-freeze                      130
el-salvador-aid                           110
religious-groups-in-schools               131
anti-satellite-test-ban                   126
aid-to-nicaraguan-contras                 128
mx-missile                                105
immigration                               108
synfuels-crporation-cutback               134
education-spending                        125
superfund-right-to-sue                    103
crime                                     112
duty-free-exports                         108
export-administration-act-south-africa    130
Name: freq, dtype: object>

In [47]:
print(
    "Training features: ", x_congress_train_processed.shape, "\n",
    "Training labels: ", y_congress_train.shape, "\n",
    "Test features: ", x_congress_test_processed.shape, "\n",
    sep=""
)

Training features: (218, 16)
Training labels: (218,)
Test features: (217, 16)



# Prediction

The following datasets ahve the following variables to use for the prediction:

- Biomed: `x_biomed_train`, `y_biomed_train`, `x_biomed_test`, `y_biomed_test`
- Fertility: `x_fert_train`, `y_fert_train`, `x_fert_test`, `y_fert_test`
- Reviews: `x_reviews_train`, `y_reviews_train`, `x_reviews_test`
- Congress: `x_congress_train`, `y_congress_train`, `x_congress_test`




## Classifier 3: Multilayer Perceptron

https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier

In [48]:

# parameters, as lists, so that I can change them more easily
activation_functions = ["logistic", "tanh", "relu"]
solvers = ["lbfgs", "sgd", "adam"]

# assigning my faetures and classes to generic names for easier switching
x_train = x_biomed_train_preprocessed
y_train = y_biomed_train

# more parameters
hidden_layers = (8,6,1)
hidden_layers = (100)

# creating the model
mlp = MLPClassifier(hidden_layer_sizes = hidden_layers, 
                    max_iter=300, # epochs
                    activation = activation_functions[1], 
                    solver=solvers[-1], 
                    random_state=123 # kind of seed
                   )
# creating a pipeline
pipe = Pipeline(steps=[('mlpc', mlp)])
# training
pipe.fit(x_train, y_train)

# printing the score (its the accuracy, that can be changed somewhere)
print(pipe.score(x_biomed_test_preprocessed, y_biomed_test))

# making predictions
y_biomed_test_pred = pipe.predict(x_biomed_test_preprocessed)
# printing out the results of my prediction vs the actual class values
for i,x in enumerate(y_biomed_test):
    print(x, "-", y_biomed_test_pred[i])

0.8571428571428571
1 - 1
0 - 0
0 - 0
0 - 0
0 - 0
1 - 0
1 - 0
0 - 0
0 - 0
1 - 1
1 - 0
0 - 0
0 - 0
0 - 0
1 - 0
0 - 0
0 - 0
1 - 1
0 - 0
1 - 1
0 - 0
1 - 0
0 - 0
1 - 1
0 - 0
0 - 0
1 - 1
0 - 0
1 - 1
0 - 0
1 - 1
0 - 0
0 - 0
0 - 0
0 - 0
0 - 0
1 - 0
0 - 0
0 - 0
0 - 0
0 - 0
1 - 1


/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


In [49]:
activation_functions = ["logistic", "tanh", "relu"]
solvers = ["lbfgs", "sgd", "adam"]
learning_rates = ["constant", "invscaling", "adaptive"]
learning_rate_init = 0.001 # default 0.001

x_train = x_biomed_train_preprocessed
y_train = y_biomed_train


hidden_layers = (8,6,1)
hidden_layers = (100)


mlp = MLPClassifier(hidden_layer_sizes = hidden_layers, 
                    max_iter=300, # epochs
                    activation = activation_functions[1], 
                    solver=solvers[-1], 
                    random_state=1234 # kind of seed
                   )

# cross validating
# first param is the model incl its params, second is features, 
# third is class labels, cv is the k of k-fold-CV, scoring is the wanted scores
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
print(cross_val_score(mlp, x_train, y_train, cv=5, scoring="recall"))
print(cross_val_score(mlp, x_train, y_train, cv=5, scoring="f1_weighted"))


/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/n

[0.75       0.91666667 0.66666667 0.75       0.83333333]


/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/n

[0.9082782  0.94117647 0.84271284 0.90562771 0.87878788]


/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


### how many hidden neurons should we use?

https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw

#### Suggestion (rule of thumb):

$\frac{N_s}{(\alpha*(N_i+N_o))}$

$N_i$ = number of input neurons

$N_o$ = number of output neurons

$N_s$ = number of samples in training data

$\alpha$ = scaling factor $\in [2,10]$



### MLP: biomed

In [50]:
x_train

Age_of_patient        ml        m2        m3        m4
0          0.317073  0.010236  0.494048  0.095775  0.252688
1          0.243902  0.012598  0.488095  0.111737  0.282258
2          0.536585  0.047244  1.000000  0.112676  0.392473
3          0.048780  0.003150  0.613095  0.107981  0.211022
4          0.292683  0.367717  0.589286  0.652582  0.849462
..              ...       ...       ...       ...       ...
162        0.780488  0.140945  0.684524  0.203756  0.456989
163        0.121951  0.013386  0.440476  0.049765  0.322581
164        0.243902  0.017323  0.613095  0.086385  0.362903
165        0.463415  0.107087  0.827381  0.167136  0.618280
166        0.390244  0.017323  0.460714  0.032864  0.174731

[167 rows x 5 columns]

In [51]:
# wie viele layers und wie groß?

mlp = MLPClassifier(hidden_layer_sizes = (2), 
                                        max_iter=500, # epochs
                                        activation = "relu", 
                                        solver="lbfgs", 
                                        #learning_rate = learning_rate,
                                        #learning_rate_init = learning_rate_init,
                                        random_state=1234 # kind of seed
                                       )
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="balanced_accuracy").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="f1_weighted").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="recall_weighted").mean())
print()

mlp = MLPClassifier(hidden_layer_sizes = (4), 
                                        max_iter=500, # epochs
                                        activation = "relu", 
                                        solver="lbfgs", 
                                        #learning_rate = learning_rate,
                                        #learning_rate_init = learning_rate_init,
                                        random_state=1234 # kind of seed
                                       )
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="balanced_accuracy").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="f1_weighted").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="recall_weighted").mean())
print()

mlp = MLPClassifier(hidden_layer_sizes = (6), 
                                        max_iter=500, # epochs
                                        activation = "relu", 
                                        solver="lbfgs", 
                                        #learning_rate = learning_rate,
                                        #learning_rate_init = learning_rate_init,
                                        random_state=1234 # kind of seed
                                       )
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="balanced_accuracy").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="f1_weighted").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="recall_weighted").mean())
print()

mlp = MLPClassifier(hidden_layer_sizes = (4, 2), 
                                        max_iter=500, # epochs
                                        activation = "relu", 
                                        solver="lbfgs", 
                                        #learning_rate = learning_rate,
                                        #learning_rate_init = learning_rate_init,
                                        random_state=1234 # kind of seed
                                       )
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="balanced_accuracy").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="f1_weighted").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="recall_weighted").mean())
print()

mlp = MLPClassifier(hidden_layer_sizes = (4, 3), 
                                        max_iter=500, # epochs
                                        activation = "relu", 
                                        solver="lbfgs", 
                                        #learning_rate = learning_rate,
                                        #learning_rate_init = learning_rate_init,
                                        random_state=1234 # kind of seed
                                       )
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="balanced_accuracy").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="f1_weighted").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="recall_weighted").mean())
print()


mlp = MLPClassifier(hidden_layer_sizes = (6, 2), 
                                        max_iter=500, # epochs
                                        activation = "logistic", 
                                        solver="lbfgs", 
                                        #learning_rate = learning_rate,
                                        #learning_rate_init = learning_rate_init,
                                        random_state=1234 # kind of seed
                                       )
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="balanced_accuracy").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="f1_weighted").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="recall_weighted").mean())
print()

mlp = MLPClassifier(hidden_layer_sizes = (6, 4), 
                                        max_iter=500, # epochs
                                        activation = "relu", 
                                        solver="lbfgs", 
                                        #learning_rate = learning_rate,
                                        #learning_rate_init = learning_rate_init,
                                        random_state=1234 # kind of seed
                                       )
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="balanced_accuracy").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="f1_weighted").mean())
print(cross_val_score(mlp, x_train, y_train, cv=10, scoring="recall_weighted").mean())
print()



0.8622727272727273
0.8774547773758974
0.8805147058823529

0.8577272727272728
0.8718309893991683
0.8746323529411765



/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.8756060606060606


/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.8899547773758973


...

0.8639393939393939


/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/nikolaus/Documents/Uni/TU/Machine Learning/

0.8831721620238244


/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/nikolaus/Documents/Uni/TU/Machine Learning/1_dataset_description/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/nikolaus/Documents/Uni/TU/Machine Learning/

0.8871323529411764

0.8925757575757576
0.9078433107103183
0.9102941176470589



#### CV

In [52]:
# performing cross-fold validation on the training data to evaluate best parameters

biomed_results = pd.DataFrame(columns=["classifier", "balanced_accuracy", 
                                       "f1_weighted", "recall", 
                                       "precision", "time_taken", "activation_function", 
                                       "solver", "learning_rate", "hidden_layer"])

# the scores we want
scoring = {'balanced_accuracy': 'balanced_accuracy',
           'f1_weighted': 'f1_weighted',
           'precision_weighted': 'precision_weighted',
           'recall_weighted': 'recall_weighted'}

# filter out warnings (not sure if this is a good idea)
warnings.filterwarnings('ignore')

activation_functions = ["logistic", "tanh", "relu"]
solvers = ["lbfgs", "sgd", "adam"]
learning_rates = ["constant", "invscaling", "adaptive"]

x_train = x_biomed_train_preprocessed
y_train = y_biomed_train

#hidden_layers = (8,6,1)
#hidden_layers = (100)
hidden_layers = [
    (2),
    (4),
    (6),
    (4, 2),
    (4, 4),
    (6, 2),
    (6, 4),
    (6, 4, 2),
]

# running index
i = 0

# cross-validation k
k = 10


# only do the computation, if the following flag is true
# it takes an hour or so to compute this, not necessary
if False:
    # iterate through all parameter permutations
    # save accuracy, f1, precisiona and recall
    for activation_function in activation_functions:
        for solver in solvers:
            for learning_rate in learning_rates:
                for hidden_layer in hidden_layers:
                    
                    # make a model
                    mlp = MLPClassifier(hidden_layer_sizes = hidden_layer, 
                                        max_iter=300, # epochs
                                        activation = activation_function, 
                                        solver=solver, 
                                        learning_rate = learning_rate,
                                        learning_rate_init = learning_rate_init,
                                        random_state=1234 # kind of seed
                                       )

                    # cross_validate() returns a dictionary with results
                    # cv = k for f-fold cross-validation
                    cv_results = cross_validate(mlp, x_train, y_train, cv=k, scoring=scoring, return_train_score=True)
                    
                    # extract the result values and take the mean of the k iterations
                    fit_time = cv_results["fit_time"].mean()
                    balanced_accuracy = cv_results["test_balanced_accuracy"].mean()
                    f1_weighted = cv_results["test_f1_weighted"].mean()
                    recall = cv_results["test_recall_weighted"].mean()
                    precision = cv_results["test_precision_weighted"].mean()
              
                    # print out the running number, the accuracy and the parameters
                    print(i, "- acc:", balanced_accuracy, "-, time:",  fit_time, end="\r")


                    # save everything
                    biomed_results = pd.concat([biomed_results, pd.DataFrame({
                        "classifier": "mlp",
                        "balanced_accuracy": balanced_accuracy,
                        "f1_weighted": [f1_weighted],
                        "recall": [recall],
                        "precision": [precision],
                        "time_taken": fit_time,
                        "activation_function": activation_function,
                        "solver": solver,
                        "learning_rate": learning_rate,
                        "hidden_layer": str(hidden_layer)                        
                    })], ignore_index=True)
                    i += 1;
    
    biomed_results
    
    # saving the results as a pickle
    with open('data/biomed/biomed_results.pickle', 'wb') as handle:
        pickle.dump(biomed_results, handle)


In [53]:
biomed_results.to_csv("data/biomed/biomed_results.csv")

In [54]:
# loading the saved results
with open('data/biomed/biomed_results.pickle', 'rb') as handle:
    biomed_results = pickle.load(handle)

display(biomed_results.sort_values("balanced_accuracy", ascending=False).head(10))
display(biomed_results.sort_values("recall", ascending=False).head(10))

classifier  balanced_accuracy  f1_weighted    recall  precision  \
158        mlp           0.892576     0.907843  0.910294   0.919991   
166        mlp           0.892576     0.907843  0.910294   0.919991   
150        mlp           0.892576     0.907843  0.910294   0.919991   
23         mlp           0.890455     0.912551  0.916544   0.926143   
7          mlp           0.890455     0.912551  0.916544   0.926143   
15         mlp           0.890455     0.912551  0.916544   0.926143   
5          mlp           0.886364     0.906814  0.911029   0.917648   
21         mlp           0.886364     0.906814  0.911029   0.917648   
13         mlp           0.886364     0.906814  0.911029   0.917648   
86         mlp           0.884848     0.896403  0.899632   0.917579   

     time_taken activation_function solver learning_rate hidden_layer  
158    0.085247                relu  lbfgs    invscaling       (6, 4)  
166    0.082983                relu  lbfgs      adaptive       (6, 4)  
150    0.084893                relu  lbfgs      constant       (6, 4)  
23     0.151449            logistic  lbfgs      adaptive    (6, 4, 2)  
7      0.148197            logistic  lbfgs      constant    (6, 4, 2)  
15     0.153861            logistic  lbfgs    invscaling    (6, 4, 2)  
5      0.117270            logistic  lbfgs      constant       (6, 2)  
21     0.125265            logistic  lbfgs      adaptive       (6, 2)  
13     0.121541            logistic  lbfgs    invscaling       (6, 2)  
86     0.133092                tanh  lbfgs    invscaling       (6, 4)

classifier  balanced_accuracy  f1_weighted    recall  precision  \
23         mlp           0.890455     0.912551  0.916544   0.926143   
15         mlp           0.890455     0.912551  0.916544   0.926143   
7          mlp           0.890455     0.912551  0.916544   0.926143   
72         mlp           0.878788     0.905071  0.911029   0.924786   
80         mlp           0.878788     0.905071  0.911029   0.924786   
5          mlp           0.886364     0.906814  0.911029   0.917648   
88         mlp           0.878788     0.905071  0.911029   0.924786   
21         mlp           0.886364     0.906814  0.911029   0.917648   
13         mlp           0.886364     0.906814  0.911029   0.917648   
166        mlp           0.892576     0.907843  0.910294   0.919991   

     time_taken activation_function solver learning_rate hidden_layer  
23     0.151449            logistic  lbfgs      adaptive    (6, 4, 2)  
15     0.153861            logistic  lbfgs    invscaling    (6, 4, 2)  
7      0.148197            logistic  lbfgs      constant    (6, 4, 2)  
72     0.057864                tanh  lbfgs      constant            2  
80     0.062135                tanh  lbfgs    invscaling            2  
5      0.117270            logistic  lbfgs      constant       (6, 2)  
88     0.059005                tanh  lbfgs      adaptive            2  
21     0.125265            logistic  lbfgs      adaptive       (6, 2)  
13     0.121541            logistic  lbfgs    invscaling       (6, 2)  
166    0.082983                relu  lbfgs      adaptive       (6, 4)

In [55]:
# bad example
# this is the model with the highest recall
# obviously it go there by just saying everyone has muscular dystrophy lol
x_train = x_biomed_train_preprocessed
y_train = y_biomed_train
# make a model
mlp = MLPClassifier(hidden_layer_sizes = (4), 
                    max_iter=300, # epochs
                    activation = "logistic", 
                    solver="sgd", 
                    learning_rate = "invscaling",
                    random_state=1234 # kind of seed
                   )

print("f1:", cross_val_score(mlp, x_train, y_train, cv=10, scoring="f1_weighted").mean())
print("recall", cross_val_score(mlp, x_train, y_train, cv=10, scoring="recall").mean())

mlp.fit(x_train, y_train)

pd.crosstab(y_biomed_test, mlp.predict(x_biomed_test_preprocessed), colnames=["predicted"], rownames=["actual"])

f1: 0.19026389211811207
recall 1.0


predicted   1
actual       
1          15
0          27

In [56]:
# visualize

x_train = x_biomed_train_preprocessed
y_train = y_biomed_train

mlp = MLPClassifier(hidden_layer_sizes = (6, 4, 2), 
                    max_iter=500, # epochs
                    activation = "logistic", 
                    solver="lbfgs", 
                    verbose=True,
                     learning_rate_init=.001,

                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)

mlp.score(x_train, y_train)
mlp.loss_

#fig, axes = plt.subplots(2, 2, figsize=(15, 10))


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           77     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.81197D-01    |proj g|=  1.17790D-01

At iterate    1    f=  6.52992D-01    |proj g|=  5.17792D-04

At iterate    2    f=  6.52991D-01    |proj g|=  3.08558D-04

At iterate    3    f=  6.52981D-01    |proj g|=  9.28675D-04

At iterate    4    f=  6.52962D-01    |proj g|=  2.30968D-03

At iterate    5    f=  6.52905D-01    |proj g|=  4.84661D-03

At iterate    6    f=  6.52726D-01    |proj g|=  9.13231D-03

At iterate    7    f=  6.49557D-01    |proj g|=  8.05513D-03

At iterate    8    f=  6.49516D-01    |proj g|=  2.27553D-02

At iterate    9    f=  6.38260D-01    |proj g|=  2.34352D-02

At iterate   10    f=  6.30874D-01    |proj g|=  6.08264D-02

At iterate   11    f=  5.20699D-01    |proj g|=  1.23179D-01

At iterate   12    f=  5.20131D-01    |proj g|=  1.30946D-01

At iterate   13    f=  4.1

 This problem is unconstrained.


-01    |proj g|=  2.84189D-03

At iterate   95    f=  1.82271D-01    |proj g|=  2.11433D-03

At iterate   96    f=  1.82211D-01    |proj g|=  5.37546D-03

At iterate   97    f=  1.82129D-01    |proj g|=  5.84879D-03

At iterate   98    f=  1.82103D-01    |proj g|=  1.12551D-02

At iterate   99    f=  1.81963D-01    |proj g|=  1.95619D-02

At iterate  100    f=  1.81941D-01    |proj g|=  4.25585D-03

At iterate  101    f=  1.81892D-01    |proj g|=  3.58800D-03

At iterate  102    f=  1.81832D-01    |proj g|=  3.57263D-03

At iterate  103    f=  1.81775D-01    |proj g|=  1.23247D-02

At iterate  104    f=  1.81651D-01    |proj g|=  4.34341D-03

At iterate  105    f=  1.81551D-01    |proj g|=  1.67673D-02

At iterate  106    f=  1.81506D-01    |proj g|=  6.12692D-03

At iterate  107    f=  1.81456D-01    |proj g|=  3.08793D-03

At iterate  108    f=  1.81432D-01    |proj g|=  3.09173D-03

At iterate  109    f=  1.81402D-01    |proj g|=  5.75400D-03

At iterate  110    f=  1.81363D-01    |

0.17488151710303415

#### best parameters & testing on holdout set
the disease is very rare, in fact, few patients in the sample even have it. We shouldd therefore aim to minimize false negatives, so we do not miss anyone having it. Therefore, we want to maximize recall.

in training, the learning rate never made a difference

In [57]:
x_train = x_biomed_train_preprocessed
y_train = y_biomed_train
x_test = x_biomed_test_preprocessed
y_test = y_biomed_test


print("best accuracy, 3d best recall")
mlp = MLPClassifier(hidden_layer_sizes = (6, 4), 
                    max_iter=300, # epochs
                    activation = "relu", 
                    solver="lbfgs", 
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = mlp.predict(x_test)
#pd.crosstab(y_biomed_test, y_pred, colnames=["predicted"], rownames=["actual"])
print("(6, 4)", "relu", "lbfgs")
print("Accuracy balanced:", balanced_accuracy_score(y_test, y_pred))
print("Recall balanced:", recall_score(y_test, y_pred, average="weighted"))
print("------")


print("best recall, 2nd best accuracy")
mlp = MLPClassifier(hidden_layer_sizes = (6, 4, 2), 
                    max_iter=300, # epochs
                    activation = "logistic", 
                    solver="lbfgs", 
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = mlp.predict(x_test)
#pd.crosstab(y_biomed_test, y_pred, colnames=["predicted"], rownames=["actual"])
print("(6, 4, 2)", "logistic", "lbfgs")
print("Accuracy balanced:", balanced_accuracy_score(y_test, y_pred))
print("Recall balanced:", recall_score(y_test, y_pred, average="weighted"))
print("------")


print("3d best accuracy")
mlp = MLPClassifier(hidden_layer_sizes = (6, 2), 
                    max_iter=300, # epochs
                    activation = "logistic", 
                    solver="lbfgs", 
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = mlp.predict(x_test)
#pd.crosstab(y_biomed_test, y_pred, colnames=["predicted"], rownames=["actual"])
print("(6, 2)", "logistic", "lbfgs")
print("Accuracy balanced:", balanced_accuracy_score(y_test, y_pred))
print("Recall balanced:", recall_score(y_test, y_pred, average="weighted"))
print("------")


print("2nd best recall")
mlp = MLPClassifier(hidden_layer_sizes = (2), 
                    max_iter=300, # epochs
                    activation = "tanh", 
                    solver="lbfgs", 
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = mlp.predict(x_test)
#pd.crosstab(y_biomed_test, y_pred, colnames=["predicted"], rownames=["actual"])
print("(2)", "tanh", "lbfgs")
print("Accuracy balanced:", balanced_accuracy_score(y_test, y_pred))
print("Recall balanced:", recall_score(y_test, y_pred, average="weighted"))
print("------")


best accuracy, 3d best recall

At iterate  305    f=  1.75671D-01    |proj g|=  1.89624D-03

At iterate  306    f=  1.75630D-01    |proj g|=  4.08641D-03

At iterate  307    f=  1.75555D-01    |proj g|=  8.62967D-04

At iterate  308    f=  1.75533D-01    |proj g|=  1.99795D-03

At iterate  309    f=  1.75485D-01    |proj g|=  3.35670D-03

At iterate  310    f=  1.75444D-01    |proj g|=  3.13307D-03

At iterate  311    f=  1.75397D-01    |proj g|=  3.56955D-03

At iterate  312    f=  1.75298D-01    |proj g|=  8.51299D-04

At iterate  313    f=  1.75258D-01    |proj g|=  3.25341D-03

At iterate  314    f=  1.75218D-01    |proj g|=  5.23578D-04

At iterate  315    f=  1.75203D-01    |proj g|=  3.81703D-04

At iterate  316    f=  1.75195D-01    |proj g|=  8.03955D-04

At iterate  317    f=  1.75190D-01    |proj g|=  5.94270D-04

At iterate  318    f=  1.75181D-01    |proj g|=  7.11230D-04

At iterate  319    f=  1.75161D-01    |proj g|=  4.23975D-04

At iterate  320    f=  1.75130D-01    |

In [58]:
# other good example
x_train = x_biomed_train_preprocessed
y_train = y_biomed_train
# make a model
mlp = MLPClassifier(hidden_layer_sizes = (4), 
                    max_iter=300, # epochs
                    activation = "tanh", 
                    solver="sgd", 
                    learning_rate = "invscaling",
                    random_state=1234 # kind of seed
                   )
print("f1:", cross_val_score(mlp, x_train, y_train, cv=10, scoring="f1_weighted").mean())
print("recall", cross_val_score(mlp, x_train, y_train, cv=10, scoring="recall").mean())

mlp.fit(x_train, y_train)
print(np.array([y_biomed_test, mlp.predict(x_biomed_test_preprocessed)]))

print()
# confusion matrix
pd.crosstab(y_biomed_test, mlp.predict(x_biomed_test_preprocessed), colnames=["predicted"], rownames=["actual"])

f1: 0.5388797219292576
recall 0.9333333333333333
[[1 0 0 0 0 1 1 0 0 1 1 0 0 0 1 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0 1 0 0 0 0 0
  1 0 0 0 0 1]
 [1 0 1 1 1 1 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 0 1 0 0 0
  1 1 1 1 0 1]]



predicted   0   1
actual           
1           0  15
0          13  14

### MLP: fertility
class labels: 0 is "normal", 1 is "altered"

intuition for evaluation: it's maybe better to falsely inform people that they are fertile, even though they may not be, than the other way around, telling potentially fertile people they aren't fertile. People failing while trying for a baby is common and typically less of a big deal than people accidentally getting pregnant believing they are unable to and also not wanting to. In other words, unwanted pregnancy with our model at fault would be the bigger issue than giving false hopes. On the other hand, the more realistic setting of this model in action is that it would advise anyone, where fertility could potentially be altered to go see a physician, so the model would not ever be blamed for an unwanted pregnancy anyways.
"Normal fertility" corresponds with 0 in our class labels, therefore avoiding mistakingly classifying a patient as having "altered fertility" means avoiding false positives, therefore we aim for high precision over recall. 


In [59]:
activation_functions = ["logistic", "tanh", "relu"]
solvers = ["lbfgs", "sgd", "adam"]
learning_rates = ["constant", "invscaling", "adaptive"]
learning_rate_init = 0.001 # default 0.001

x_train = x_fert_train
y_train = y_fert_train


hidden_layers = (8,6,1)
hidden_layers = (21)


mlp = MLPClassifier(hidden_layer_sizes = hidden_layers, 
                    max_iter=300, # epochs
                    activation = activation_functions[1], 
                    solver=solvers[-1], 
                    random_state=1234 # kind of seed
                   )

# cross validating
# first param is the model incl its params, second is features, 
# third is class labels, cv is the k of k-fold-CV, scoring is the wanted scores
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
print(cross_val_score(mlp, x_train, y_train, cv=5, scoring="accuracy"))
print(cross_val_score(mlp, x_train, y_train, cv=5, scoring="f1_weighted"))


[0.9375 0.875  0.875  0.8125 0.75  ]
[0.92816092 0.81666667 0.81666667 0.78448276 0.75      ]


#### CV

In [60]:
# testing params for fertility
fertility_results = pd.DataFrame(columns=["classifier", "balanced_accuracy", 
                                       "f1_weighted", "recall", 
                                       "precision", "time_taken", "activation_function", 
                                       "solver", "learning_rate", 
                                       "hidden_layer"])
# the scores we want
scoring = {'balanced_accuracy': 'balanced_accuracy',
           'f1_weighted': 'f1_weighted',
           'precision_weighted': 'precision_weighted',
           'recall_weighted': 'recall_weighted'}

# performing cross-fold validation on the training data to evaluate best parameters

# filter out warnings (not sure if this is a good idea)
warnings.filterwarnings('ignore')

activation_functions = ["logistic", "tanh", "relu"]
solvers = ["lbfgs", "sgd", "adam"]
learning_rates = ["constant", "invscaling", "adaptive"]

x_train = x_fert_train
y_train = y_fert_train

hidden_layers = [
    (2),
    (10),
    (21),
    (10, 5),
    (10, 10),
    (21, 2),
    (21, 10),
    (21, 10, 2),
]

# cross-validation
k = 5

# running number to count iterations/permutations
i = 0


# only do the computation, if the following flag is true
# it takes an hour or so to compute this, not necessary
if False:
    # iterate through all parameter permutations
    # save accuracy, f1, precisiona and recall
    for activation_function in activation_functions:
        for solver in solvers:
            for learning_rate in learning_rates:
                for hidden_layer in hidden_layers:
                    
                    # make a model
                    mlp = MLPClassifier(hidden_layer_sizes = hidden_layer, 
                                        max_iter=300, # epochs
                                        activation = activation_function, 
                                        solver=solver, 
                                        learning_rate = learning_rate,
                                        learning_rate_init = learning_rate_init,
                                        random_state=1234 # kind of seed
                                       )

                    # cross_validate() returns a dictionary with results
                    # cv = k for f-fold cross-validation
                    cv_results = cross_validate(mlp, x_train, y_train, cv=k, scoring=scoring, return_train_score=True)
                    
                    # extract the result values and take the mean of the 5 iterations
                    fit_time = cv_results["fit_time"].mean()
                    balanced_accuracy = cv_results["test_balanced_accuracy"].mean()
                    f1_weighted = cv_results["test_f1_weighted"].mean()
                    recall = cv_results["test_recall_weighted"].mean()
                    precision = cv_results["test_precision_weighted"].mean()
                    
                    # print out the running number, the accuracy and the parameters
                    print(i, "- acc:", balanced_accuracy, "-, time:",  fit_time, end="\r")
                    
                    # save everything
                    fertility_results = pd.concat([fertility_results, pd.DataFrame({
                        "classifier": "mlp",
                        "balanced_accuracy": [balanced_accuracy],
                        "f1_weighted": [f1_weighted],
                        "recall": [recall],
                        "precision": [precision],
                        "time_taken": fit_time,
                        "activation_function": activation_function,
                        "solver": solver,
                        "learning_rate": learning_rate,
                        "hidden_layer": str(hidden_layer)                        
                    })], ignore_index=True)
                    i += 1;
 
    display(fertility_results)
    
    # saving the results as a pickle
    with open('data/fertility/fertility_results_unscaled.pickle', 'wb') as f:
        pickle.dump(fertility_results, f)


In [61]:
fertility_results[fertility_results.hidden_layer=="2"]

Empty DataFrame
Columns: [classifier, balanced_accuracy, f1_weighted, recall, precision, time_taken, activation_function, solver, learning_rate, hidden_layer]
Index: []

First training run mit k=10

Wow, the results so far are pretty bad. F1 scores are looking ok-ish, but accuracy never even reaches 0.7. Precision is quite low, meaning we get heaps of false positives. Upon crying over these results, we remembered that the training data only contains 80 samples, so doing 10-fold cross-validation might have been a mistake. Now rerunning with k=5.

In [62]:
with open('data/fertility/fertility_results_k5.pickle', 'rb') as f:
        fertility_results = pickle.load(f)

display(fertility_results.sort_values("balanced_accuracy", ascending=False).head(10))
display(fertility_results.sort_values("f1_weighted", ascending=False).head(10))

classifier  balanced_accuracy  f1_weighted  recall  precision  time_taken  \
32         mlp           0.700000     0.747264  0.7000   0.858598    0.011627   
110        mlp           0.635714     0.652808  0.5875   0.838694    0.115512   
0          mlp           0.600000     0.817196  0.8250   0.816330    0.039305   
8          mlp           0.600000     0.817196  0.8250   0.816330    0.027684   
16         mlp           0.600000     0.817196  0.8250   0.816330    0.027929   
19         mlp           0.592857     0.807928  0.8125   0.813754    0.035992   
11         mlp           0.592857     0.807928  0.8125   0.813754    0.042255   
3          mlp           0.592857     0.807928  0.8125   0.813754    0.037799   
179        mlp           0.585714     0.278273  0.2750   0.893571    0.079978   
199        mlp           0.571429     0.830862  0.8500   0.818125    0.127086   

    activation_function solver learning_rate hidden_layer  
32             logistic    sgd    invscaling            2  
110                tanh    sgd    invscaling     (21, 10)  
0              logistic  lbfgs      constant            2  
8              logistic  lbfgs    invscaling            2  
16             logistic  lbfgs      adaptive            2  
19             logistic  lbfgs      adaptive      (10, 5)  
11             logistic  lbfgs    invscaling      (10, 5)  
3              logistic  lbfgs      constant      (10, 5)  
179                relu    sgd    invscaling      (10, 5)  
199                relu   adam      constant  (21, 10, 2)

classifier  balanced_accuracy  f1_weighted  recall  precision  time_taken  \
210        mlp           0.550000     0.838966  0.8875   0.800833    0.099973   
202        mlp           0.550000     0.838966  0.8875   0.800833    0.100424   
194        mlp           0.550000     0.838966  0.8875   0.800833    0.098712   
215        mlp           0.571429     0.830862  0.8500   0.818125    0.125675   
142        mlp           0.571429     0.830862  0.8500   0.818125    0.127235   
207        mlp           0.571429     0.830862  0.8500   0.818125    0.128002   
199        mlp           0.571429     0.830862  0.8500   0.818125    0.127086   
126        mlp           0.571429     0.830862  0.8500   0.818125    0.125987   
127        mlp           0.571429     0.830862  0.8500   0.818125    0.146762   
134        mlp           0.571429     0.830862  0.8500   0.818125    0.129492   

    activation_function solver learning_rate hidden_layer  
210                relu   adam      adaptive           21  
202                relu   adam    invscaling           21  
194                relu   adam      constant           21  
215                relu   adam      adaptive  (21, 10, 2)  
142                tanh   adam      adaptive     (21, 10)  
207                relu   adam    invscaling  (21, 10, 2)  
199                relu   adam      constant  (21, 10, 2)  
126                tanh   adam      constant     (21, 10)  
127                tanh   adam      constant  (21, 10, 2)  
134                tanh   adam    invscaling     (21, 10)

accuracy is still isnt ascending over 0.7, but we got way higher precision this time


#### best params & testing
here the learning rate DID make a difference



In [63]:
x_train = x_fert_train
y_train = y_fert_train
x_test = x_fert_test
y_test = y_fert_test

print("best accuracy")
mlp = MLPClassifier(hidden_layer_sizes = (2), 
                    max_iter=300, # epochs
                    activation = "logistic", 
                    solver="sgd", 
                    learning_rate="invscaling",
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = mlp.predict(x_test)
#pd.crosstab(y_biomed_test, y_pred, colnames=["predicted"], rownames=["actual"])
print("(2)", "logistic", "sgd", "invscaling learning rate")
print("Accuracy balanced:", balanced_accuracy_score(y_test, y_pred))
print("F1 balanced:", f1_score(y_test, y_pred, average="weighted"))
print("Precision balanced:", precision_score(y_test, y_pred, average="weighted"))
print("------")


print("best f1 weighted and best precision")
mlp = MLPClassifier(hidden_layer_sizes = (21), 
                    max_iter=300, # epochs
                    activation = "relu", 
                    solver="adam", 
                  random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = mlp.predict(x_test)
#pd.crosstab(y_biomed_test, y_pred, colnames=["predicted"], rownames=["actual"])
print("(21)", "relu", "adam")
print("Accuracy balanced:", balanced_accuracy_score(y_test, y_pred))
print("F1 balanced:", f1_score(y_test, y_pred, average="weighted"))
print("Precision balanced:", precision_score(y_test, y_pred, average="weighted"))
print("------")





best accuracy
(2) logistic sgd invscaling learning rate
Accuracy balanced: 0.5833333333333333
F1 balanced: 0.7189964157706094
Precision balanced: 0.845054945054945
------
best f1 weighted and best precision
(21) relu adam
Accuracy balanced: 0.4444444444444444
F1 balanced: 0.8
Precision balanced: 0.8
------


### MLP: reviews
class labels are not binary, so we cannot use f1, recall or precision but rather have to rely on weighted accuracy

following our rules of thumb, we should use few hidden layers with nodes between 50 and 10,000


In [64]:
x_train

age  child_diseases  accident  surgery  hours_sitting  fall  spring  \
35  0.78               0         0        1           0.38     0       0   
13  0.81               0         1        1           0.38     1       0   
21  0.75               0         1        1           0.25     1       0   
26  0.67               0         1        0           0.38     1       0   
50  0.67               0         1        0           0.19     0       1   
..   ...             ...       ...      ...            ...   ...     ...   
7   1.00               0         0        0           0.38     0       1   
10  0.67               0         0        1           0.31     1       0   
45  0.53               0         1        1           0.44     0       0   
17  0.69               0         1        0           0.25     1       0   
80  0.92               0         0        1           0.63     0       1   

    summer  winter  fever_never  fever_not_recent  fever_recent  \
35       0       1            1                 0             0   
13       0       0            0                 1             0   
21       0       0            0                 1             0   
26       0       0            0                 1             0   
50       0       0            0                 1             0   
..     ...     ...          ...               ...           ...   
7        0       0            0                 1             0   
10       0       0            0                 0             1   
45       0       1            1                 0             0   
17       0       0            0                 1             0   
80       0       0            1                 0             0   

    smoking_daily  smoking_never  smoking_occasionally  alcohol_daily  \
35              0              1                     0              0   
13              0              1                     0              0   
21              0              0                     1              0   
26              0              1                     0              0   
50              0              1                     0              0   
..            ...            ...                   ...            ...   
7               0              1                     0              0   
10              0              0                     1              0   
45              0              0                     1              0   
17              0              1                     0              0   
80              0              1                     0              0   

    alcohol_rarely_or_never  alcohol_several_daily  alcohol_several_weekly  \
35                        0                      0                       1   
13                        1                      0                       0   
21                        0                      0                       1   
26                        0                      0                       1   
50                        0                      0                       0   
..                      ...                    ...                     ...   
7                         0                      0                       1   
10                        0                      0                       0   
45                        1                      0                       0   
17                        0                      0                       0   
80                        1                      0                       0   

    alcohol_weekly  
35               0  
13               0  
21               0  
26               0  
50               1  
..             ...  
7                0  
10               1  
45               0  
17               1  
80               0  

[80 rows x 20 columns]

In [65]:
# grid search

# testing params for reviews
reviews_results = pd.DataFrame(columns=["classifier", "balanced_accuracy", 
                                       "time_taken", "activation_function", 
                                       "solver", "learning_rate", 
                                       "hidden_layer", "iterations"])
# the scores we want
# no point in checking for precision or recall
# arguably, one may be worse than the other, but not for the sake of the research :D
scoring = {'balanced_accuracy': 'balanced_accuracy'}

# performing cross-fold validation on the training data to evaluate best parameters

# filter out warnings (not sure if this is a good idea)
warnings.filterwarnings('ignore')

activation_function = "logistic"
solvers = ["lbfgs", "sgd", "adam"]
solvers = ["adam"]
learning_rates = ["constant"]

x_train = x_reviews_train
y_train = y_reviews_train





hidden_layers = [
    (50, 50, 50),
    (200, 100, 50),
    (1000, 1000),
    (1000, 1000, 50),
    (2000, 500, 50),
#    (5000),
#    (1000),
#    (10000),
]

# cross-validation
k = 10

# running number to count iterations/permutations
i = 0


# only do the computation, if the following flag is true
# it takes an hour or so to compute this, not necessary
if False:
    # iterate through all parameter permutations
    # save accuracy, f1, precisiona and recall
    for max_iter in [500, 800]:
        for solver in solvers:
            for learning_rate in learning_rates:
                for hidden_layer in hidden_layers:
                    
                    # make a model
                    mlp = MLPClassifier(hidden_layer_sizes = hidden_layer, 
                                        max_iter=max_iter, # epochs
                                        activation = activation_function, 
                                        solver=solver, 
                                        verbose=True,
                                        learning_rate = learning_rate,
                                        learning_rate_init = learning_rate_init,
                                        random_state=1234 # kind of seed
                                       )

                    # cross_validate() returns a dictionary with results
                    # cv = k for f-fold cross-validation
                    cv_results = cross_validate(mlp, x_train, y_train, cv=k, scoring=scoring, return_train_score=True)
                    
                    # extract the result values and take the mean of the k iterations
                    fit_time = cv_results["fit_time"].mean()
                    balanced_accuracy = cv_results["test_balanced_accuracy"].mean()
                    
                    # print out the running number, the accuracy and the parameters
                    print(i, "- acc:", balanced_accuracy, "-, time:",  fit_time, end="\r")
                    
                    # save everything
                    reviews_results = pd.concat([reviews_results, pd.DataFrame({
                        "classifier": "mlp",
                        "balanced_accuracy": [balanced_accuracy],
                        "time_taken": fit_time,
                        "activation_function": activation_function,
                        "solver": solver,
                        "iterations": max_iter,
                        "learning_rate": learning_rate,
                        "hidden_layer": str(hidden_layer)                        
                    })], ignore_index=True)
                    i += 1;
 
    display(reviews_results)
    
    # saving the results as a pickle
    with open(f'data/reviews/reviews_results_k{k}_4.pickle', 'wb') as f:
        pickle.dump(reviews_results, f)


In [66]:
# loading the saved results
with open('data/reviews/reviews_results_k10.pickle', 'rb') as handle:
    reviews_results_1 = pickle.load(handle)
display(reviews_results_1.sort_values("balanced_accuracy", ascending=False))

# loading the saved results
with open('data/reviews/reviews_results_k10_2.pickle', 'rb') as handle:
    reviews_results_2 = pickle.load(handle)
display(reviews_results_2.sort_values("balanced_accuracy", ascending=False))

# loading the saved results
with open('data/reviews/reviews_results_k10_3.pickle', 'rb') as handle:
    reviews_results_3 = pickle.load(handle)
display(reviews_results_3.sort_values("balanced_accuracy", ascending=False))

# loading the saved results
with open('data/reviews/reviews_results_k10_4.pickle', 'rb') as handle:
    reviews_results_4 = pickle.load(handle)
display(reviews_results_4.sort_values("balanced_accuracy", ascending=False))

classifier  balanced_accuracy  time_taken activation_function solver  \
1        mlp           0.740408  101.882982            logistic   adam   
3        mlp           0.686245   67.672742                tanh   adam   
5        mlp           0.657619   69.126942                relu   adam   
0        mlp           0.611830   63.347642            logistic   adam   
2        mlp           0.505810   42.704505                tanh   adam   
4        mlp           0.251490   43.051773                relu   adam   

  learning_rate hidden_layer  
1      constant         1000  
3      constant         1000  
5      constant         1000  
0      constant           50  
2      constant           50  
4      constant           50

classifier  balanced_accuracy  time_taken activation_function solver  \
2        mlp           0.611830   71.385762            logistic   adam   
5        mlp           0.611830   84.956902            logistic   adam   
4        mlp           0.497667  204.181449            logistic    sgd   
1        mlp           0.454939  152.379840            logistic    sgd   
0        mlp           0.164449  132.227634            logistic  lbfgs   
3        mlp           0.161449  165.899050            logistic  lbfgs   

  learning_rate hidden_layer  
2      constant           50  
5      constant           50  
4      constant           50  
1      constant           50  
0      constant           50  
3      constant           50

classifier  balanced_accuracy  time_taken activation_function solver  \
0        mlp           0.688497  166.714588            logistic   adam   
1        mlp           0.688497  150.525065            logistic   adam   

  learning_rate  hidden_layer iterations  
0      constant  (1000, 1000)        500  
1      constant  (1000, 1000)        800

classifier  balanced_accuracy   time_taken activation_function solver  \
2        mlp           0.688497   126.050256            logistic   adam   
7        mlp           0.688497   145.618657            logistic   adam   
4        mlp           0.463415  1628.143984            logistic   adam   
9        mlp           0.463415  1611.926593            logistic   adam   
3        mlp           0.423150   576.500537            logistic   adam   
8        mlp           0.423150   581.137089            logistic   adam   
1        mlp           0.348694   207.914831            logistic   adam   
6        mlp           0.348694   227.487648            logistic   adam   
0        mlp           0.266429   105.945277            logistic   adam   
5        mlp           0.261408   131.797761            logistic   adam   

  learning_rate      hidden_layer iterations  
2      constant      (1000, 1000)        500  
7      constant      (1000, 1000)        800  
4      constant   (2000, 500, 50)        500  
9      constant   (2000, 500, 50)        800  
3      constant  (1000, 1000, 50)        500  
8      constant  (1000, 1000, 50)        800  
1      constant    (200, 100, 50)        500  
6      constant    (200, 100, 50)        800  
0      constant      (50, 50, 50)        500  
5      constant      (50, 50, 50)        800

#### what are the best params??

In [67]:
# save results for kaggle
x_train = x_reviews_train_preprocessed
y_train = y_reviews_train


x_test = x_reviews_test_preprocessed
y_test = y_fert_test

print("model 1")
mlp = MLPClassifier(hidden_layer_sizes = (50), 
                    max_iter=500, # epochs
                    activation = "logistic", # acc to sklearn logistic is best for big datasets
                    solver="adam", 
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = pd.DataFrame({"ID": reviews_test_ID, "Class": mlp.predict(x_test)})
y_pred.to_csv("data/reviews/reviews_pred_mlp1.csv", index=False)
print("model 1: logistic, adam, (50)")



print("model 2")
mlp = MLPClassifier(hidden_layer_sizes = (500), 
                    max_iter=500, # epochs
                    activation = "logistic", # acc to sklearn logistic is best for big datasets
                    solver="adam", 
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = pd.DataFrame({"ID": reviews_test_ID, "Class": mlp.predict(x_test)})
y_pred.to_csv("data/reviews/reviews_pred_mlp2.csv", index=False)
print("model 2: logistic, adam, (500)")
print("model 3")
mlp = MLPClassifier(hidden_layer_sizes = (5000), 
                    max_iter=500, # epochs
                    activation = "logistic", # acc to sklearn logistic is best for big datasets
                    solver="adam", 
                    verbose=True,
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = pd.DataFrame({"ID": reviews_test_ID, "Class": mlp.predict(x_test)})
y_pred.to_csv("data/reviews/reviews_pred_mlp3.csv", index=False)
print("model 3: logistic, adam, (5000)")
print("model 4")
mlp = MLPClassifier(hidden_layer_sizes = (1000, 1000), 
                    max_iter=500, # epochs
                    activation = "logistic", # acc to sklearn logistic is best for big datasets
                    solver="adam", 
                    #verbose=True,
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = pd.DataFrame({"ID": reviews_test_ID, "Class": mlp.predict(x_test)})
y_pred.to_csv("data/reviews/reviews_pred_mlp4.csv", index=False)
print("model 4: logistic, adam, (1000, 1000)")
print("model 5")
mlp = MLPClassifier(hidden_layer_sizes = (2000, 500, 50), 
                    max_iter=500, # epochs
                    activation = "logistic", # acc to sklearn logistic is best for big datasets
                    solver="adam", 
                    #verbose=True,
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = pd.DataFrame({"ID": reviews_test_ID, "Class": mlp.predict(x_test)})
y_pred.to_csv("data/reviews/reviews_pred_mlp5.csv", index=False)
print("model 5: logistic, adam, (2000, 500, 50)")

model 1


### MLP: congress
we go from 16 features to 2 labels

rule of thumb suggests to use anything between 2 and 6 hidden layers

In [68]:
# recoding the labels to be 0 or 1
# 0 is democrat
# 1 is repulican
y_congress_train.replace({"democrat": 0, "republican": 1}, inplace=True)
# this way I can reuse my classification code from earlier

In [69]:
display(x_congress_train_processed)
display(y_congress_train)
display(x_congress_test_processed)


handicapped-infants  water-project-cost-sharing  \
0                    0.0                         1.0   
1                    1.0                         0.0   
2                    0.0                         0.0   
3                    0.0                         1.0   
4                    1.0                         0.0   
..                   ...                         ...   
213                  0.0                         1.0   
214                  0.0                         1.0   
215                  0.0                         0.0   
216                  1.0                         1.0   
217                  0.0                         1.0   

     adoption-of-the-budget-resolution  physician-fee-freeze  el-salvador-aid  \
0                                  1.0                   0.0              0.0   
1                                  0.0                   1.0              1.0   
2                                  0.0                   1.0              1.0   
3                                  1.0                   0.0              0.0   
4                                  1.0                   1.0              1.0   
..                                 ...                   ...              ...   
213                                1.0                   0.0              0.0   
214                                1.0                   0.0              1.0   
215                                0.0                   1.0              1.0   
216                                1.0                   0.0              1.0   
217                                0.0                   1.0              1.0   

     religious-groups-in-schools  anti-satellite-test-ban  \
0                            0.0                      1.0   
1                            0.0                      1.0   
2                            1.0                      0.0   
3                            1.0                      1.0   
4                            1.0                      1.0   
..                           ...                      ...   
213                          1.0                      0.0   
214                          1.0                      0.0   
215                          1.0                      0.0   
216                          1.0                      0.0   
217                          1.0                      0.0   

     aid-to-nicaraguan-contras  mx-missile  immigration  \
0                          1.0         0.0          0.0   
1                          0.0         0.0          1.0   
2                          0.0         0.0          1.0   
3                          1.0         1.0          0.0   
4                          1.0         0.0          1.0   
..                         ...         ...          ...   
213                        1.0         1.0          1.0   
214                        1.0         0.0          1.0   
215                        0.0         0.0          0.0   
216                        1.0         0.0          1.0   
217                        0.0         0.0          1.0   

     synfuels-crporation-cutback  education-spending  superfund-right-to-sue  \
0                            1.0                 0.0                     0.0   
1                            0.0                 0.0                     0.0   
2                            0.0                 1.0                     0.0   
3                            1.0                 1.0                     0.0   
4                            1.0                 1.0                     0.0   
..                           ...                 ...                     ...   
213                          0.0                 0.0                     1.0   
214                          1.0                 0.0                     1.0   
215                          0.0                 1.0                     1.0   
216                          1.0                 0.0                     1.0   
217                          0.0     

100    0
215    1
139    1
178    0
15     1
      ..
106    0
14     0
92     1
179    0
102    1
Name: class, Length: 218, dtype: int64

handicapped-infants  water-project-cost-sharing  \
0                    1.0                         0.0   
1                    0.0                         0.0   
2                    1.0                         1.0   
3                    1.0                         1.0   
4                    1.0                         0.0   
..                   ...                         ...   
212                  0.0                         0.0   
213                  1.0                         0.0   
214                  0.0                         0.0   
215                  1.0                         0.0   
216                  0.0                         1.0   

     adoption-of-the-budget-resolution  physician-fee-freeze  el-salvador-aid  \
0                                  1.0                   0.0              1.0   
1                                  0.0                   1.0              1.0   
2                                  1.0                   0.0              0.0   
3                                  1.0                   0.0              0.0   
4                                  1.0                   0.0              1.0   
..                                 ...                   ...              ...   
212                                1.0                   0.0              0.0   
213                                1.0                   0.0              0.0   
214                                1.0                   1.0              1.0   
215                                1.0                   0.0              0.0   
216                                0.0                   1.0              1.0   

     religious-groups-in-schools  anti-satellite-test-ban  \
0                            1.0                      1.0   
1                            1.0                      0.0   
2                            0.0                      1.0   
3                            1.0                      1.0   
4                            1.0                      0.0   
..                           ...                      ...   
212                          0.0                      1.0   
213                          0.0                      1.0   
214                          1.0                      1.0   
215                          0.0                      1.0   
216                          1.0                      0.0   

     aid-to-nicaraguan-contras  mx-missile  immigration  \
0                          0.0         0.0          0.0   
1                          0.0         0.0          1.0   
2                          1.0         1.0          0.0   
3                          1.0         1.0          1.0   
4                          0.0         0.0          0.0   
..                         ...         ...          ...   
212                        1.0         1.0          1.0   
213                        1.0         1.0          1.0   
214                        1.0         0.0          1.0   
215                        1.0         1.0          0.0   
216                        0.0         0.0          0.0   

     synfuels-crporation-cutback  education-spending  superfund-right-to-sue  \
0                            1.0                 0.0                     0.0   
1                            0.0                 1.0                     1.0   
2                            1.0                 0.0                     0.0   
3                            0.0                 0.0                     0.0   
4                            0.0                 0.0                     0.0   
..                           ...                 ...                     ...   
212                          1.0                 0.0                     0.0   
213                          0.0                 0.0                     0.0   
214                          0.0                 0.0                     0.0   
215                          0.0                 0.0                     0.0   
216                          0.0     

In [70]:
activation_functions = ["logistic", "tanh", "relu"]
solvers = ["lbfgs", "sgd", "adam"]
learning_rates = ["constant", "invscaling", "adaptive"]
learning_rate_init = 0.001 # default 0.001

x_train = x_congress_train_processed
y_train = y_congress_train


hidden_layers = (16) # best so far (16, 16, 2)

mlp = MLPClassifier(hidden_layer_sizes = hidden_layers, 
                    max_iter=500, # epochs
                    activation = activation_functions[1], 
                    solver=solvers[0], 
                    # verbose=True, # show progress
                    random_state=1234 # kind of seed
                   )

# cross validating
# first param is the model incl its params, second is features, 
# third is class labels, cv is the k of k-fold-CV, scoring is the wanted scores
# https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
print(cross_val_score(mlp, x_train, y_train, cv=5, scoring="accuracy").mean())



0.9495771670190274


In [71]:
# grid search

# testing params for congress
congress_results = pd.DataFrame(columns=["classifier", "balanced_accuracy", 
                                       "time_taken", "activation_function", 
                                       "solver", "learning_rate", 
                                       "hidden_layer"])
# the scores we want
# no point in checking for precision or recall
# arguably, one may be worse than the other, but not for the sake of the research :D
scoring = {'balanced_accuracy': 'balanced_accuracy'}

# performing cross-fold validation on the training data to evaluate best parameters

# filter out warnings (not sure if this is a good idea)
warnings.filterwarnings('ignore')

activation_functions = ["logistic", "tanh", "relu"]
solvers = ["lbfgs", "sgd", "adam"]
learning_rates = ["constant", "invscaling", "adaptive"]

x_train = x_congress_train_processed
y_train = y_congress_train




hidden_layers = [
    (2),
    (8),
    (16),
    (2, 2),
    (8, 2),
    (16, 8),
    (16, 8, 2),
    (16, 16, 8),
    (16, 16, 2),
    (16, 16, 16, 2),
    (16, 16, 16, 16)
]

# cross-validation
k = 10

# running number to count iterations/permutations
i = 0


# only do the computation, if the following flag is true
# it takes an hour or so to compute this, not necessary
if False:
    # iterate through all parameter permutations
    # save accuracy, f1, precisiona and recall
    for activation_function in activation_functions:
        for solver in solvers:
            for learning_rate in learning_rates:
                for hidden_layer in hidden_layers:
                    
                    # make a model
                    mlp = MLPClassifier(hidden_layer_sizes = hidden_layer, 
                                        max_iter=300, # epochs
                                        activation = activation_function, 
                                        solver=solver, 
                                        learning_rate = learning_rate,
                                        learning_rate_init = learning_rate_init,
                                        random_state=1234 # kind of seed
                                       )

                    # cross_validate() returns a dictionary with results
                    # cv = k for f-fold cross-validation
                    cv_results = cross_validate(mlp, x_train, y_train, cv=k, scoring=scoring, return_train_score=True)
                    
                    # extract the result values and take the mean of the k iterations
                    fit_time = cv_results["fit_time"].mean()
                    balanced_accuracy = cv_results["test_balanced_accuracy"].mean()
                    
                    # print out the running number, the accuracy and the parameters
                    print(i, "- acc:", balanced_accuracy, "-, time:",  fit_time, end="\r")
                    
                    # save everything
                    congress_results = pd.concat([congress_results, pd.DataFrame({
                        "classifier": "mlp",
                        "balanced_accuracy": [balanced_accuracy],
                        "time_taken": fit_time,
                        "activation_function": activation_function,
                        "solver": solver,
                        "learning_rate": learning_rate,
                        "hidden_layer": str(hidden_layer)                        
                    })], ignore_index=True)
                    i += 1;
 
    display(congress_results)
    
    # saving the results as a pickle
    with open('data/congress/congress_results_k10.pickle', 'wb') as f:
        pickle.dump(congress_results, f)


In [72]:
# loading the saved results
with open('data/congress/congress_results_k10.pickle', 'rb') as handle:
    congress_results = pickle.load(handle)

display(congress_results.sort_values("balanced_accuracy", ascending=False).head(30))

classifier  balanced_accuracy  time_taken activation_function solver  \
10         mlp           0.975893    0.047264            logistic  lbfgs   
32         mlp           0.975893    0.084928            logistic  lbfgs   
21         mlp           0.975893    0.044197            logistic  lbfgs   
16         mlp           0.973214    0.014908            logistic  lbfgs   
5          mlp           0.973214    0.014848            logistic  lbfgs   
27         mlp           0.973214    0.014618            logistic  lbfgs   
24         mlp           0.969643    0.008914            logistic  lbfgs   
2          mlp           0.969643    0.009134            logistic  lbfgs   
13         mlp           0.969643    0.009614            logistic  lbfgs   
112        mlp           0.962225    0.014998                tanh  lbfgs   
101        mlp           0.962225    0.013293                tanh  lbfgs   
123        mlp           0.962225    0.016805                tanh  lbfgs   
292        mlp           0.961134    0.334190                relu   adam   
270        mlp           0.961134    0.233120                relu   adam   
281        mlp           0.961134    0.208312                relu   adam   
219        mlp           0.960165    0.031877                relu  lbfgs   
208        mlp           0.960165    0.031319                relu  lbfgs   
230        mlp           0.960165    0.049969                relu  lbfgs   
198        mlp           0.959272    0.011989                relu  lbfgs   
220        mlp           0.959272    0.011509                relu  lbfgs   
209        mlp           0.959272    0.011919                relu  lbfgs   
184        mlp           0.959150    0.216131                tanh   adam   
195        mlp           0.959150    0.205934                tanh   adam   
173        mlp           0.959150    0.224672                tanh   adam   
221        mlp           0.957563    0.013147                relu  lbfgs   
210        mlp           0.957563    0.011757                relu  lbfgs   
199        mlp           0.957563    0.011605                relu  lbfgs   
99         mlp           0.955701    0.030122                tanh  lbfgs   
110        mlp           0.955701    0.021125                tanh  lbfgs   
121        mlp           0.955701    0.023767                tanh  lbfgs   

    learning_rate      hidden_layer  
10       constant  (16, 16, 16, 16)  
32       adaptive  (16, 16, 16, 16)  
21     invscaling  (16, 16, 16, 16)  
16     invscaling           (16, 8)  
5        constant           (16, 8)  
27       adaptive           (16, 8)  
24       adaptive                16  
2        constant                16  
13     invscaling                16  
112    invscaling                16  
101      constant                16  
123      adaptive                16  
292      adaptive        (16, 8, 2)  
270      constant        (16, 8, 2)  
281    invscaling        (16, 8, 2)  
219    invscaling  (16, 16, 16, 16)  
208      constant  (16, 16, 16, 16)  
230      adaptive  (16, 16, 16, 16)  
198      constant                 2  
220      adaptive                 2  
209    invscaling                 2  
184    invscaling       (16, 16, 2)  
195      adaptive       (16, 16, 2)  
173      constant       (16, 16, 2)  
221      adaptive                 8  
210    invscaling                 8  
199      constant                 8  
99       constant                 2  
110    invscaling                 2  
121      adaptive                 2

#### best params & testing
learning_rate apparently makes no difference



In [210]:
x_train = x_congress_train_processed
y_train = y_congress_train


x_test = x_congress_test_processed
y_test = y_fert_test

print("best accuracy")
mlp = MLPClassifier(hidden_layer_sizes = (16, 16, 16, 16), 
                    max_iter=300, # epochs
                    activation = "logistic", 
                    solver="lbfgs", 
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = pd.DataFrame({"ID": x_congress_test_ID, "class": mlp.predict(x_test)})
y_pred["class"] = y_pred["class"].apply(lambda x: "republican" if x else "democrat")
y_pred.to_csv("data/congress/congress_pred_mlp1.csv", index=False)
print("model 1: logistic, lbfgs, (16, 16, 16, 16)")




print("2nd best accuracy")
mlp = MLPClassifier(hidden_layer_sizes = (16, 8), 
                    max_iter=300, # epochs
                    activation = "logistic", 
                    solver="lbfgs", 
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = pd.DataFrame({"ID": x_congress_test_ID, "class": mlp.predict(x_test)})
y_pred["class"] = y_pred["class"].apply(lambda x: "republican" if x else "democrat")
y_pred.to_csv("data/congress/congress_pred_mlp2.csv", index=False)
print("model 2: logistic, lbfgs, (16, 8)")





print("3rd best accuracy")
mlp = MLPClassifier(hidden_layer_sizes = (16), 
                    max_iter=300, # epochs
                    activation = "logistic", 
                    solver="lbfgs", 
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = pd.DataFrame({"ID": x_congress_test_ID, "class": mlp.predict(x_test)})
y_pred["class"] = y_pred["class"].apply(lambda x: "republican" if x else "democrat")
y_pred.to_csv("data/congress/congress_pred_mlp3.csv", index=False)
print("model 3: logistic, lbfgs, (16)")






print("4th best accuracy")
mlp = MLPClassifier(hidden_layer_sizes = (16), 
                    max_iter=300, # epochs
                    activation = "tanh", 
                    solver="lbfgs", 
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = pd.DataFrame({"ID": x_congress_test_ID, "class": mlp.predict(x_test)})
y_pred["class"] = y_pred["class"].apply(lambda x: "republican" if x else "democrat")
y_pred.to_csv("data/congress/congress_pred_mlp4.csv", index=False)
print("model 4: tanh, lbfgs, (16)")




print("5th best accuracy")
mlp = MLPClassifier(hidden_layer_sizes = (16, 8, 2), 
                    max_iter=300, # epochs
                    activation = "relu", 
                    solver="adam", 
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
y_pred = pd.DataFrame({"ID": x_congress_test_ID, "class": mlp.predict(x_test)})
y_pred["class"] = y_pred["class"].apply(lambda x: "republican" if x else "democrat")
y_pred.to_csv("data/congress/congress_pred_mlp5.csv", index=False)
print("model 5: relu, adam, (16, 8, 2)")




5th best accuracy
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          289     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  7.16475D-01    |proj g|=  1.42556D-01

At iterate    1    f=  6.64884D-01    |proj g|=  4.07722D-02

At iterate    2    f=  6.02671D-01    |proj g|=  4.18205D-02

At iterate    3    f=  2.50415D-01    |proj g|=  3.42998D-02

At iterate    4    f=  2.12430D-01    |proj g|=  4.08754D-02

At iterate    5    f=  1.90439D-01    |proj g|=  3.00679D-02

At iterate    6    f=  1.24454D-01    |proj g|=  1.33285D-02

At iterate    7    f=  8.23757D-02    |proj g|=  8.00820D-03

At iterate    8    f=  4.68276D-02    |proj g|=  5.60973D-03

At iterate    9    f=  2.28652D-02    |proj g|=  3.11850D-03

At iterate   10    f=  1.26397D-02    |proj g|=  4.44356D-03

At iterate   11    f=  7.15799D-03    |proj g|=  2.09117D-03

At iterate   12    f=  4.81236D-03    |proj g|=  1.04736D-03

At itera

 This problem is unconstrained.


In [74]:
x_congress_test

handicapped-infants  water-project-cost-sharing  \
0                    1.0                         0.0   
1                    0.0                         0.0   
2                    1.0                         1.0   
3                    1.0                         1.0   
4                    1.0                         0.0   
..                   ...                         ...   
212                  NaN                         0.0   
213                  1.0                         0.0   
214                  0.0                         0.0   
215                  1.0                         0.0   
216                  0.0                         NaN   

     adoption-of-the-budget-resolution  physician-fee-freeze  el-salvador-aid  \
0                                  1.0                   0.0              1.0   
1                                  0.0                   1.0              1.0   
2                                  1.0                   0.0              0.0   
3                                  1.0                   0.0              0.0   
4                                  1.0                   0.0              1.0   
..                                 ...                   ...              ...   
212                                1.0                   0.0              0.0   
213                                1.0                   0.0              0.0   
214                                1.0                   1.0              1.0   
215                                1.0                   0.0              0.0   
216                                0.0                   1.0              1.0   

     religious-groups-in-schools  anti-satellite-test-ban  \
0                            1.0                      1.0   
1                            1.0                      0.0   
2                            0.0                      1.0   
3                            1.0                      1.0   
4                            1.0                      0.0   
..                           ...                      ...   
212                          0.0                      1.0   
213                          0.0                      1.0   
214                          1.0                      1.0   
215                          0.0                      1.0   
216                          1.0                      0.0   

     aid-to-nicaraguan-contras  mx-missile  immigration  \
0                          0.0         0.0          0.0   
1                          0.0         0.0          1.0   
2                          1.0         1.0          0.0   
3                          1.0         1.0          1.0   
4                          0.0         0.0          0.0   
..                         ...         ...          ...   
212                        1.0         1.0          1.0   
213                        1.0         1.0          1.0   
214                        1.0         0.0          1.0   
215                        1.0         1.0          0.0   
216                        0.0         0.0          0.0   

     synfuels-crporation-cutback  education-spending  superfund-right-to-sue  \
0                            1.0                 0.0                     0.0   
1                            0.0                 1.0                     1.0   
2                            1.0                 0.0                     0.0   
3                            0.0                 0.0                     NaN   
4                            0.0                 0.0                     0.0   
..                           ...                 ...                     ...   
212                          1.0                 NaN                     0.0   
213                          0.0                 0.0                     0.0   
214                          0.0                 0.0                     0.0   
215                          0.0                 0.0                     0.0   
216                          0.0     

In [75]:
x_train = x_congress_train_processed
y_train = y_congress_train


x_test = x_congress_test_processed
y_test = y_fert_test


mlp = MLPClassifier(hidden_layer_sizes = (8), 
                    max_iter=500, # epochs
                    activation = "logistic", 
                    solver="lbfgs", 
                  #  verbose=True,
                    random_state=1234 # kind of seed
                   )
mlp.fit(x_train, y_train)
#y_pred = pd.DataFrame({"ID": x_congress_test_ID, "class": mlp.predict(x_test)})
#y_pred["class"] = y_pred["class"].apply(lambda x: "republican" if x else "democrat")
#y_pred.to_csv("data/congress/congress_pred_mlp5.csv", index=False)
mlp.loss_


0.000594898348559961